# 811s
### outputs
* Gets 811 DDHTR vlook ups - Named: DDHTR 811 Vlookup info

In [2]:
import pandas as pd
import os
import glob
import numpy as np
import smartsheet

#Variables
# random
excel = '.xlsx'
csv = '.csv'

# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API

Central.errors_as_exceptions(True)

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)

#download smartsheets to any folder you need
# Central.Sheets.get_sheet_as_excel(Central_id, download_path=r"C:\Users\jacque.trahan\Downloads")
# South.Sheets.get_sheet_as_excel(South_id, download_path=r"C:\Users\jacque.trahan\Downloads") 


#Google Sheets
#Sheet ID comes right after /d/ in the URL
Google_sheet_ID= '1O2dmdhEo5hJVAgRVDl4M4n3_v-KqHS56q3OZVdpZANU'
# name of tab you want to use
Google_sheets_name = 'Tree RTR Tracker'

# this is tricky. after /export? we must put format=csv#id={Sheet Name}   -> manually to make this work
url = f'https://docs.google.com/spreadsheets/d/{Google_sheet_ID}/export?format=csv#id={Google_sheets_name}'

# Function to read in column titles and row vales and make out data frames
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

df_both = pd.concat([Central_SS,South_SS])

# Smartsheets uplaod below updated above. No longer need to download smartsheets

In [2]:
# path = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
# file_type = ".xlsx"
# load_it = glob.glob(path + file_type)
# df = pd.read_excel(max(load_it, key=os.path.getctime))

In [3]:
# path2 = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
# loadit2 = glob.glob(path2 + file_type)
# df2 = pd.read_excel(max(loadit2, key=os.path.getctime))

In [4]:
# df_both = pd.concat([df,df2])

# Set up vlookup for DDTHR 811s tracker

In [5]:
vlookup = df_both.copy()

vlookup = vlookup[['APN_ROW Segment','Street #', 'Street Name', 'City', 'County',
                      'Total Number of Vehicles', 'Number of Trees',"Not Eligible", 'Debris Finish', 
                      'Structural Status', 'ROE Type', '811 Called (DDHTR)', '811 Expiration Date', 'Debris Crew', 'ROE Type']]

vlookup['811 Called (DDHTR)'] = np.where(vlookup['811 Called (DDHTR)'] == 1, 'Completed', 'Incomplete')

vlookup.set_index('APN_ROW Segment', inplace=True)
vlookup.to_excel('DDHTR 811 Vlookup info.xlsx')

# Missing 811s with up coming crews

In [8]:
crew_811s = df_both.copy()
filt_out = ['FSO Returned to County', 'Erosion Control', 'FSO Failed', 'TREE ONLY ROEs', 'Ready for FSO',
           'Ineligible/Withdrawn', 'ELD Ready -Access Issues', 'Planning Hold', 'Need Site Assessment', 
           'State - Debris Eligibility Review', 'Need ASB Assessment', 'FSO Pending Propane Tank', 
           'Program Eligibility Determination', 'Nevada/Placer Pending Results', 'Returned to County', 'Punch List', 
           'Withdrawn', 'Program Eligibility Pending', ]



crew_811s = crew_811s[['APN_ROW Segment','Street #', 'Street Name', 'City', 'County', 'Debris Crew', 'Structural Status',
                      '811 Called (DDHTR)']]
crew_811s = crew_811s[(~crew_811s['Debris Crew'].isin(filt_out)) & (crew_811s['811 Called (DDHTR)'] == False)]


crew_811s.to_excel('See missing 811s.xlsx')